In [3]:
import torch
from torch import nn
import numpy as np
from chbmit import chbmit
from gnn.GraphLearner import GraphLearnerNN
from torch_geometric.utils.convert import to_networkx
from torch_geometric.nn.conv import GATv2Conv


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
patient_0 = chbmit.CHB_MIT_PAITENT(tok_len=60, ctx_len=1, path="../physionet.org/files/chbmit/1.0.0/chb01/")

In [6]:
batch_size = 16
dataloader = torch.utils.data.DataLoader(patient_0, batch_size=batch_size, shuffle=True)

In [20]:
class GraphAttentionNetwork(nn.Module):
    def __init__(self, kernel_size = 2):
        super().__init__()
        self.gsl = GraphLearnerNN(60*256,threshold = 0.04)

        self.layer1 = GATv2Conv(60*256, 30*256)

        self.layer2 = GATv2Conv(30*256, 15*256)

        self.pooling = nn.AvgPool2d(kernel_size=kernel_size)

        self.activation = torch.nn.LeakyReLU()

    def forward(self, x):

        edge_idx, edge_weights = self.gsl(x)

        edge_idx = edge_idx.to(int).to(device)

        x = torch.reshape(x, (x.shape[0] * x.shape[1], x.shape[2]))
        x = self.layer1(x, edge_idx)

        x = self.activation(x)

        x = self.layer2(x, edge_idx)
        x = self.activation(x)
        
        x = x.unsqueeze(0).unsqueeze(0)
        x = self.pooling(x)

        linear = nn.Linear(x.shape[-1] * x.shape[-2], 2)
        x = linear(torch.flatten(x))
        return x

In [21]:
input = next(iter(dataloader))[0].to(device)
GAT = GraphAttentionNetwork().to(device)
output = GAT(input)
print(output)

OutOfMemoryError: CUDA out of memory. Tried to allocate 450.00 MiB. GPU 0 has a total capacity of 7.58 GiB of which 193.06 MiB is free. Including non-PyTorch memory, this process has 6.36 GiB memory in use. Of the allocated memory 5.83 GiB is allocated by PyTorch, and 350.03 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)